In [ ]:
latest_gameweek = 1
shift_param = 1

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
import os
import datetime as dt
import pickle
import json
import requests

from sklearn.linear_model import LogisticRegression
from scipy.stats import poisson

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
# model for estimating bonus points based on gameweek bps
model_path = Path(f"../models/logistic_regression_for_bonus_points.pkl")
with open(model_path, "rb") as f:
    clf = pickle.load(f)

# Functions

In [ ]:
def fpl_data_processing(df, columns):

    xg_data = []
    xa_data = []
    xga_data = []
    for ix, row in df.iterrows():
        my_gameweek = row['gameweek']
        xg_data.append( row[f'xG_week{my_gameweek}'] )
        xa_data.append( row[f'xA_week{my_gameweek}'] )
        xga_data.append( row[f'xGA_week{my_gameweek}'] )

    df['gameweek_xG'] = xg_data
    df['gameweek_xA'] = xa_data
    df['gameweek_xGA'] = xga_data

    df_new = df[columns].copy()

    return df_new

In [ ]:
def my_fill_na(x, gameweek_col, diff_col):
    '''Fill nan values for first items for grouped variables where diff is calculated. But also don't fill for season 22-23,
    where data is missing for a number of weeks at the beginning of the season.'''
    my_value = x[diff_col] if (np.isnan(x[gameweek_col])) & (x['minutes']<=90) else x[gameweek_col]
    return my_value
    

In [ ]:
def calculate_xPoints(x,clf):
    """Expected points for a given gameweek given underlying stats for that gameweek."""

    clean_sheet_points = np.array([4,4,1,0])
    goal_points = np.array([6,6,5,4])

    # calculate expexted points
    points_played = np.array([1 if x['gameweek_minutes']>0 else 0])
    points_played_over_60 = np.array([1 if x['gameweek_minutes']>=60 else 0])
    points_xG = goal_points[x['element_type']-1] * x['gameweek_xG']
    points_xA = x['gameweek_xA'] * 3
    clean_sheet_probability = np.array(poisson.pmf(0,x['team_xGA']))
    points_clean_sheet = [clean_sheet_points[x['element_type']-1] * clean_sheet_probability if x['gameweek_minutes']>=60 else 0]
    points_saves = x['gameweek_saves'] // 3
    points_penalty_saves = x['gameweek_penalties_saved'] * 5 * 0.21 #points for save times approx. probability of penalty save
    #penalty_for_penalty_miss = x['Performance_PKatt'] * (-2*0.21) # this data only on fbref
    # estimate bonus points
    if not np.isnan(x['gameweek_bps']):
        y_pred_prob = clf.predict_proba(np.array(x['gameweek_bps']).reshape(-1, 1))
    else:
        # return nan if bonus points can't be estimated 
        return np.nan
    points_bonus = np.matmul(y_pred_prob, np.array([0,1,2,3]).reshape((4,1)))
    
    # penalty for possible points deductions based on goals conceded
    xGA = x['team_xGA']
    # calculate penalty
    xGA_conceded_penalty = -(poisson.pmf(2,xGA)+poisson.pmf(3,xGA))-(poisson.pmf(4,xGA)+poisson.pmf(5,xGA))-(poisson.pmf(6,xGA)+poisson.pmf(7,xGA))-(poisson.pmf(8,xGA)+poisson.pmf(9,xGA)-(poisson.pmf(10,xGA)+poisson.pmf(11,xGA)))
    # apply penalty only to GK and DEF
    if (x['element_type']==1) | (x['element_type']==2):
        xGA_conceded_penalty = xGA_conceded_penalty
    else:
        xGA_conceded_penalty = 0
    # scale penalty with playing time
    xGA_conceded_penalty = (x['gameweek_minutes'] / 90) * xGA_conceded_penalty

    penalty_for_cards = [-3 if x['gameweek_red_cards']==1 else -1 if x['gameweek_yellow_cards']==1 else 0]
    penalty_for_own_goal = -2 * x['gameweek_own_goals']

    # add up all point components
    total_points = float(points_played + points_played_over_60 + points_xG + points_xA + points_clean_sheet + points_saves +\
                    points_penalty_saves + points_bonus + xGA_conceded_penalty +\
                    penalty_for_cards + penalty_for_own_goal)
    
    return total_points

# Fetch new data

In [ ]:
# teams for season 23-24
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
         'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
         'Liverpool', 'Luton', 'Manchester City', 'Manchester Utd',
         'Newcastle Utd', 'Nottingham Forest', 'Sheffield Utd', 'Tottenham',
         'West Ham', 'Wolves']

In [ ]:
# fetch FPL data online
fpl_online_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_online_df = pd.DataFrame(fpl_online_data['elements'])
fpl_online_df['team_name'] = [teams[i] for i in fpl_online_df['team']-1]
fpl_online_df['name'] = fpl_online_df.apply(lambda x: x['first_name'] + ' ' + x['second_name'], axis=1)
fpl_online_df

In [ ]:
# CREATE NEW DATA SET IF THERE IS NEW DATA AVAILABLE AND SAVE TO FILE

# fetch latest fpl data from data folder
folder_path_str = '../data/fpl/'
folder_path = Path(folder_path_str)
files = os.listdir(folder_path)
# drop non-csv files (e.g. DS_Store)
files = [file for file in files if file.endswith('.csv')]
# sort files and pick last one
files = np.sort(files)
file = files[-1]
full_path = folder_path_str + file
old_data = pd.read_csv(full_path, index_col=0)

# only take players who have played, i.e., minutes>0
new_data = fpl_online_df[fpl_online_df.minutes>0].copy()
# players who have now played but had not previously played at all
new_data_1 = new_data[~new_data.name.isin(old_data.name.unique())].copy()
# players whose minutes are higher now than previously
aux = new_data[new_data.name.isin(old_data.name.unique())].copy()
new_rows = []
for ix, row in aux.iterrows():
    player_name = row['name']
    change_in_minutes = row['minutes'] - old_data.loc[old_data.name==player_name, 'minutes'].iloc[-1]
    if change_in_minutes > 0:
        new_rows.append(row)
if len(new_rows) > 0:
    new_data_2 = pd.concat([new_rows])
else:
    new_data_2 = pd.DataFrame() # empty df

# overwrites old new_data variable
new_data = pd.concat([new_data_1, new_data_2], ignore_index=True)

# create new data set combining old and new data and save to file
if new_data.shape[0] > 0:



    # add info
    new_data['gameweek'] = latest_gameweek
    new_data['season'] = '23-24'
    time_now = dt.datetime.now()
    new_data['data_retrieved_datetime'] = time_now
    display(new_data)

    full_data = pd.concat([old_data, new_data], ignore_index=True)
    print(f'Full data shape: {full_data.shape}')
    
    # save new full data
    path = Path('../data/fpl/data_' + str(time_now.strftime("%Y%m%d-%H%M%S")) + '.csv')
    full_data.to_csv(path)